Đề bài : 

Một cửa hàng quần áo cần xây dựng cơ sở dữ liệu cho ứng dụng quản lý bán hàng online gồm có các bảng như sau:

- Bảng Category(lưu trữ các danh mục sản phẩm)
- Bảng Product(Lưu trữ các sản phẩm theo danh mục)
- Bảng Customer(Lưu trữ thông tin khách hàng)
- Bảng Orders (lưu trư các hóa đơn của khách hàng khi họ mua hàng) 
- Bảng OrderDetail(chi tiết hóa đơn)

## Kết nối tới db

In [1]:
import sqlite3

conn = sqlite3.connect('data/store.db')
c = conn.cursor()

## Tạo các bảng theo yêu cầu

In [8]:
# Tạo bảng Category
c.execute('''
CREATE TABLE IF NOT EXISTS CATEGORY (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME VARCHAR NOT NULL,
    STATUS BOOLEAN
)
''')

# Tạo bảng Product
c.execute('''
CREATE TABLE IF NOT EXISTS PRODUCT (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME TEXT NOT NULL,
    STATUS BOOLEAN,
    PRICE REAL,
    SALE_PRICE REAL,
    CREATED_DATE DATE,
    CATEGORY_ID INTEGER,
    FOREIGN KEY (CATEGORY_ID) REFERENCES CATEGORY (ID)
    
)
''')
c.execute('''
CREATE INDEX IF NOT EXISTS IDX_PRODUCT_NAME ON PRODUCT (NAME);
''')
# Tạo bảng Customer
c.execute('''
CREATE TABLE IF NOT EXISTS CUSTOMER (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    NAME TEXT NOT NULL ,
    EMAIL TEXT,
    PHONE TEXT,
    ADDRESS TEXT,
    CREATED_DATE DATE,
    GENDER BOOLEAN,
    BIRTH_DAY DATE
)
''')

# Tạo bảng Orders
c.execute('''
CREATE TABLE IF NOT EXISTS ORDERS (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    CUSTOMER_ID INTEGER NOT NULL,
    STATUS BOOLEAN,
    ORDER_DATE DATE NOT NULL,
    FOREIGN KEY (CUSTOMER_ID) REFERENCES CUSTOMER (ID)
)
''')

# Tạo bảng OrderDetail
c.execute('''
CREATE TABLE IF NOT EXISTS ORDER_DETAIL (
    ORDER_ID INTEGER NOT NULL,
    PRODUCT_ID INTEGER NOT NULL,
    QUANTITY INTEGER NOT NULL,
    PRICE REAL NOT NULL,
    FOREIGN KEY (ORDER_ID) REFERENCES ORDERS (ID),
    FOREIGN KEY (PRODUCT_ID) REFERENCES PRODUCT (ID)
)
''')

# Lưu thay đổi và đóng kết nối
conn.commit()

Thêm dữ liệu vào bảng category

In [3]:
categories = [
	("Men's Clothing", True),
	("Women's Clothing", True),
	("Kid's Clothing", True),
	("Accessories", True),
	("Shoes", True)
]

c.executemany('''
INSERT INTO CATEGORY (NAME, STATUS) VALUES (?, ?)
''', categories)

Thêm dữ liệu vào bảng product

In [4]:
products = [
	("T-Shirt", True, 19.99, 15.99, '2024-01-01', 1),
	("Jeans", True, 39.99, 29.99, '2024-01-02', 1),
	("Jacket", True, 59.99, 49.99, '2024-01-03', 1),
	("Dress", True, 49.99, 39.99, '2024-01-04', 2),
	("Skirt", True, 29.99, 19.99, '2024-01-05', 2),
	("Blouse", True, 39.99, 29.99, '2024-01-06', 2),
	("Kid's T-Shirt", True, 9.99, 7.99, '2024-01-07', 3),
	("Kid's Jeans", True, 19.99, 14.99, '2024-01-08', 3),
	("Kid's Jacket", True, 29.99, 24.99, '2024-01-09', 3),
	("Hat", True, 14.99, 9.99, '2024-01-10', 4),
	("Scarf", True, 19.99, 14.99, '2024-01-11', 4),
	("Belt", True, 24.99, 19.99, '2024-01-12', 4),
	("Sneakers", True, 49.99, 39.99, '2024-01-13', 5),
	("Boots", True, 69.99, 59.99, '2024-01-14', 5),
	("Sandals", True, 29.99, 24.99, '2024-01-15', 5)
]

c.executemany('''
INSERT INTO PRODUCT (NAME, STATUS, PRICE, SALE_PRICE, CREATED_DATE, CATEGORY_ID) VALUES (?, ?, ?, ?, ?, ?)
''', products)

Thêm dữ liệu vào bảng customer

In [5]:
customers = [
	("Alice", "alice@example.com", "1234567890", "123 Main St", '2024-01-01', True, '1990-01-01'),
	("Bob", "bob@example.com", "0987654321", "456 Elm St", '2024-01-02', False, '1985-02-02'),
	("Charlie", "charlie@example.com", "5678901234", "789 Oak St", '2024-01-03', True, '1995-03-03')
]

c.executemany('''
INSERT INTO CUSTOMER (NAME, EMAIL, PHONE, ADDRESS, CREATED_DATE, GENDER, BIRTH_DAY) VALUES (?, ?, ?, ?, ?, ?, ?)
''', customers)


Thêm dữ liệu vào bảng orders

In [9]:
orders = [
	(1, True, '2024-02-01'),
	(2, True, '2024-02-02'),
	(3, True, '2024-02-03')
]

c.executemany('''
INSERT INTO ORDERS (CUSTOMER_ID, STATUS, ORDER_DATE) VALUES (?, ?, ?)
''', orders)


Thêm dữ liệu vào bảng order_detail

In [10]:
order_details = [
	(1, 1, 2, 15.99),
	(1, 2, 1, 29.99),
	(2, 3, 1, 49.99),
	(2, 4, 2, 39.99),
	(3, 5, 1, 19.99),
	(3, 6, 2, 29.99)
]

c.executemany('''
INSERT INTO ORDER_DETAIL (ORDER_ID, PRODUCT_ID, QUANTITY, PRICE) VALUES (?, ?, ?, ?)
''', order_details)


In [11]:
conn.commit()

#### Yêu cầu truy vấn dữ liệu

1. Lấy ra danh sách sản phẩm có sắp xếp giảm dần theo Price gồm các cột sau: : Id, Name, Price, SalePrice, Status, CategoryName, CreatedDate

In [21]:
data = c.execute('''
SELECT P.ID, P.NAME, PRICE, SALE_PRICE, P.STATUS, C.NAME AS CATEGORY_NAME , P.CREATED_DATE 
FROM PRODUCT P
INNER JOIN CATEGORY C ON P.CATEGORY_ID = C.ID ORDER BY PRICE DESC 
''')
data.fetchall()

[(14, 'Boots', 69.99, 59.99, 1, 'Shoes', '2024-01-14'),
 (3, 'Jacket', 59.99, 49.99, 1, "Men's Clothing", '2024-01-03'),
 (4, 'Dress', 49.99, 39.99, 1, "Women's Clothing", '2024-01-04'),
 (13, 'Sneakers', 49.99, 39.99, 1, 'Shoes', '2024-01-13'),
 (2, 'Jeans', 39.99, 29.99, 1, "Men's Clothing", '2024-01-02'),
 (6, 'Blouse', 39.99, 29.99, 1, "Women's Clothing", '2024-01-06'),
 (5, 'Skirt', 29.99, 19.99, 1, "Women's Clothing", '2024-01-05'),
 (9, "Kid's Jacket", 29.99, 24.99, 1, "Kid's Clothing", '2024-01-09'),
 (15, 'Sandals', 29.99, 24.99, 1, 'Shoes', '2024-01-15'),
 (12, 'Belt', 24.99, 19.99, 1, 'Accessories', '2024-01-12'),
 (1, 'T-Shirt', 19.99, 15.99, 1, "Men's Clothing", '2024-01-01'),
 (8, "Kid's Jeans", 19.99, 14.99, 1, "Kid's Clothing", '2024-01-08'),
 (11, 'Scarf', 19.99, 14.99, 1, 'Accessories', '2024-01-11'),
 (10, 'Hat', 14.99, 9.99, 1, 'Accessories', '2024-01-10'),
 (7, "Kid's T-Shirt", 9.99, 7.99, 1, "Kid's Clothing", '2024-01-07')]

2. lấy ra danh sách Category gồm: Id, Name, TotalProduct, Status

In [22]:
data = c.execute('''
SELECT C.ID, C.NAME, C.STATUS, COUNT(P.ID)AS TOTAL_PRODUCT
FROM PRODUCT P
INNER JOIN CATEGORY C ON P.CATEGORY_ID = C.ID
GROUP BY P.CATEGORY_ID
''')
data.fetchall()

[(1, "Men's Clothing", 1, 3),
 (2, "Women's Clothing", 1, 3),
 (3, "Kid's Clothing", 1, 3),
 (4, 'Accessories', 1, 3),
 (5, 'Shoes', 1, 3)]

3. Truy vấn danh sách Customer gồm: Id, Name, Email, Phone, Address,
CreatedDate, Gender, BirthDay, Age (Age là cột suy ra từ BirthDay, Gender nếu = 0
là Nam, 1 là Nữ ) ( Age = năm hiện tại - năm sinh của BirthDay)


In [10]:
data = c.execute('''
SELECT ID,NAME,EMAIL,PHONE, ADDRESS, CREATED_DATE,
CASE WHEN GENDER = 1 THEN 'Male'
ELSE 'Female' END AS GENDER,
 CAST((STRFTIME('%Y', 'NOW') - STRFTIME('%Y', BIRTH_DAY))
 - (STRFTIME('%m-%d', 'NOW') < STRFTIME('%m-%d', BIRTH_DAY))
 AS INTEGER) AS AGE
FROM CUSTOMER C

''')
data.fetchall()

[(1,
  'Alice',
  'alice@example.com',
  '1234567890',
  '123 Main St',
  '2024-01-01',
  'Male',
  34),
 (2,
  'Bob',
  'bob@example.com',
  '0987654321',
  '456 Elm St',
  '2024-01-02',
  'Female',
  39),
 (3,
  'Charlie',
  'charlie@example.com',
  '5678901234',
  '789 Oak St',
  '2024-01-03',
  'Male',
  29)]

4. Truy vấn xóa các sản phẩm chưa được bán 

In [26]:
# đầu tiên hãy commit tất cả các transaction bên trên đã
conn.commit()

- cách 1: xóa cứng (dùng câu lệnh delete )

In [ ]:
c.execute('''
DELETE FROM PRODUCTS P 
WHERE 
ID NOT IN (SELECT PRODUCT_ID FROM ORDERS_DETAIL GROUP BY PRODUCT_ID) 
''')

In [35]:
conn.rollback()

- cách 2: xóa mềm (dùng câu lệnh update )

In [34]:
data=c.execute('''
UPDATE PRODUCT 
SET STATUS = 0
WHERE 
ID NOT IN (SELECT PRODUCT_ID FROM ORDER_DETAIL GROUP BY PRODUCT_ID) 
''')
data.fetchall()

[]

In [33]:
conn.commit()

5. Cập nhật Cột sale_price tăng thêm 10% cho tất cả các sản phẩm có sale price <= 250000

In [40]:
c.execute('''
UPDATE PRODUCT 
SET SALE_PRICE = SALE_PRICE + SALE_PRICE/100*10
WHERE SALE_PRICE <= 250000
''')
conn.commit()